In [10]:
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

In [27]:
file_path1 = r"G:\fly\dataframe_data\classification.csv"
file_path2 = r"G:\code\BrainParcellation-main\BrainParcellation-main\microenviron\data\mefeatures_100K.csv"
file_path3 = r"G:\fly\dataframe_data\proj_ccf-me_test_copy.csv"
file_path4 = r"G:\fly\dataframe_data\proj_test_copy.csv"
SER_path = r"G:\fly\dataframe_data\SER.csv"
ACH_path = r"G:\fly\dataframe_data\ACH.csv"
DA_path = r"G:\fly\dataframe_data\DA.csv"
GABA_path = r"G:\fly\dataframe_data\GABA.csv"
GLUT_path = r"G:\fly\dataframe_data\GLUT.csv"
OCT_path = r"G:\fly\dataframe_data\OCT.csv"

df = pd.read_csv(file_path1)
me_feature_df = pd.read_csv(file_path2)

visual_df = df[df['class'] == 'visual']
visual_df1 = visual_df[visual_df['cell_type'] == 'R1-6']
visual_df1['function'] = 'R1-6'
visual_df2 = visual_df[visual_df['cell_type'] == 'ocellar_retinula_cell']
visual_df2['function'] = 'ocellar_retinula_cell'
visual_df3 = visual_df[visual_df['cell_type'] == 'R7']
visual_df3['function'] = 'R7'
visual_df4 = visual_df[visual_df['cell_type'] == 'R8']
visual_df4['function'] = 'R8'
olfactory_df = df[df['class'] == 'olfactory']
olfactory_df['function'] = 'olfactory'
mechano_df = df[df['class'] == 'mechanosensory']
mechano_df['function'] = 'mechanosensory'
gustatory_df = df[df['class'] == 'gustatory']
gustatory_df['function'] = 'gustatory'
hygros_df = df[df['class'] == 'hygrosensory']
hygros_df['function'] = 'hygrosensory'
thermo_df = df[df['class'] == 'thermosensory']
thermo_df['function'] = 'thermosensory'
motor_df = df[df['super_class'] == 'motor']
motor_df['function'] = 'motor'
ocellar_df = df[df['super_class'] == 'optic']
ocellar_df['function'] = 'optic'


#visual_df, olfactory_df, mechano_df, gustatory_df, hygros_df, thermo_df, motor_df
combined_df = pd.concat([visual_df1, visual_df2, visual_df3, visual_df4, olfactory_df, mechano_df, gustatory_df, hygros_df, thermo_df, motor_df])
merged_df = pd.merge(combined_df[['Name','function']], me_feature_df, on='Name', how='left')
merged_df = merged_df.dropna()

me_columns = [col for col in merged_df.columns if col.endswith('_me')]
single_columns = [item.replace('_me', '') for item in me_columns]
#ccf_me_columns = proj_ccf_me_df.select_dtypes(include=['number']).columns.tolist()
#ccf_me_columns = [col for col in ccf_me_columns if col != 'Name']
#ccf_columns = proj_ccf_df.select_dtypes(include=['number']).columns.tolist()
#ccf_columns = [col for col in ccf_columns if col != 'Name']
#ccf-me
# merged_df2 = pd.merge(combined_df[['Name','function']], proj_ccf_me_df, on='Name', how='left')
# merged_df2 = merged_df2.dropna()
#ccf
# merged_df3 = pd.merge(combined_df[['Name','function']], proj_ccf_df, on='Name', how='left')
# merged_df3 = merged_df3.dropna()

In [12]:
combined_df['function'].value_counts()

function
R1-6                     8452
mechanosensory           2648
olfactory                2281
R7                       1342
R8                       1324
gustatory                 343
ocellar_retinula_cell     273
motor                     106
hygrosensory               74
thermosensory              29
Name: count, dtype: int64

In [13]:
merged_df['function'].value_counts()

function
R1-6                     5477
olfactory                1422
R7                       1020
R8                        954
mechanosensory            778
ocellar_retinula_cell     233
gustatory                 130
motor                     106
hygrosensory               49
thermosensory              29
Name: count, dtype: int64

In [35]:
import plotly.express as px
import umap
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

type = 'single'

if type=="me":
    df_features = merged_df[me_columns + ['function']]
    scaler = StandardScaler()
    df_features_scaled = scaler.fit_transform(df_features[me_columns])
else:
    df_features = merged_df[single_columns + ['function']]
    scaler = StandardScaler()
    df_features_scaled = scaler.fit_transform(df_features[single_columns])

umap_model = umap.UMAP(
    n_neighbors=30,         
    min_dist=0.1,           
    n_components=2,         
    metric='euclidean',     
    learning_rate=0.3,      
    spread=1,               
    random_state=None,      
    set_op_mix_ratio=1,       
    local_connectivity=1,     
    angular_rp_forest=False,  
    init='spectral',        
    verbose=False           
)


umap_features = umap_model.fit_transform(df_features_scaled)

df_features['UMAP1'] = umap_features[:, 0]
df_features['UMAP2'] = umap_features[:, 1]
df_features[['function', 'UMAP1', 'UMAP2']].to_csv(r'G:\manuscript\figure5\single_UMAP.csv', index=False)

